## **Importing the dependencies**

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from datetime import datetime

SEED = 1234
tf.random.set_seed(SEED)  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Data Loading & Preprocessing**

In [ ]:
apply_data_augmentation = True

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=30,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        brightness_range=[0.4,1.0],
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255)

# Create validation and test ImageDataGenerator objects
valid_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Create generators to read images from dataset directory
# -------------------------------------------------------
dataset_dir = os.path.join('/content/drive/My Drive', 'MaskDataset')

# Batch size
bs = 16

# img shape
img_h = 380
img_w = 380

num_classes=3

decide_class_indices = False
if decide_class_indices:
    classes = ['NoMask',        # 0
               'AllMask'        # 1
               'SomeMask']      # 2
else:
    classes=None

# Training
training_dir = os.path.join(dataset_dir, 'training')
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               batch_size=bs,
                                               classes=classes,
                                               class_mode='categorical',
                                               target_size=(img_h, img_w),
                                               shuffle=True,
                                               seed=SEED)  # targets are directly converted into one-hot vectors

# Validation
validation_dir = os.path.join(dataset_dir, 'validation')
valid_gen = valid_data_gen.flow_from_directory(validation_dir,
                                               batch_size=bs, 
                                               classes=classes,
                                               class_mode='categorical',
                                               target_size=(img_h, img_w),
                                               shuffle=False,
                                               seed=SEED)

# Test
test_dir = os.path.join(dataset_dir, 'test')
test_gen = test_data_gen.flow_from_directory(test_dir,
                                             batch_size=bs, 
                                             classes=classes,
                                             class_mode='categorical',
                                             target_size=(img_h, img_w),
                                             shuffle=False,
                                             seed=SEED)

Found 4717 images belonging to 3 classes.
Found 897 images belonging to 3 classes.
Found 0 images belonging to 0 classes.


In [ ]:
# Create Dataset objects
# ----------------------

# Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

valid_dataset = valid_dataset.repeat()

# Test
# ----
test_dataset = tf.data.Dataset.from_generator(lambda: test_gen,
                                              output_types=(tf.float32, tf.float32),
                                              output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
test_dataset = valid_dataset.repeat()

## **Model**

In [ ]:
# Architecture: Features extraction -> Classifier

start_f = 8
depth = 3

model = tf.keras.Sequential()
model.add(tf.keras.layers.Cropping2D(cropping=(90, 90), data_format=None))


# Features extraction
for i in range(depth):

    if i == 0:
        input_shape = [img_h, img_w, 3]
    else:
        input_shape=[None]

    # Conv block: Conv2D -> Activation -> Pooling
    model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                     kernel_size=(3, 3),
                                     strides=(1, 1),
                                     padding='same',
                                     activation='relu',
                                     input_shape=input_shape))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

    start_f *= 2

 
# Classifier
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(units=256, activation='relu'))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [ ]:
# Visualize created model as a table
model.summary()

In [ ]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
exps_dir = os.path.join('/content/drive/My Drive/Keras3/', 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
    callbacks.append(es_callback)

# ReduceLROnPlateau
# --------------

lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                                   patience=5, min_lr=0.00001)

callbacks.append(lr_callback)

## **Training**

In [ ]:
model.fit(x=train_dataset,
          epochs=100,  
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

Epoch 1/100
295/295 [==============================] - 1126s 4s/step - loss: 1.1105 - accuracy: 0.3392 - val_loss: 1.0967 - val_accuracy: 0.3534
Epoch 2/100
295/295 [==============================] - 190s 645ms/step - loss: 1.0962 - accuracy: 0.3653 - val_loss: 1.0954 - val_accuracy: 0.4047
Epoch 3/100
295/295 [==============================] - 182s 616ms/step - loss: 1.0935 - accuracy: 0.3755 - val_loss: 1.0729 - val_accuracy: 0.4526
Epoch 4/100
295/295 [==============================] - 177s 601ms/step - loss: 1.0720 - accuracy: 0.4257 - val_loss: 1.0090 - val_accuracy: 0.4961
Epoch 5/100
295/295 [==============================] - 176s 595ms/step - loss: 1.0293 - accuracy: 0.4745 - val_loss: 1.0179 - val_accuracy: 0.4738
Epoch 6/100
295/295 [==============================] - 175s 594ms/step - loss: 0.9859 - accuracy: 0.5001 - val_loss: 0.9889 - val_accuracy: 0.5184
Epoch 7/100
295/295 [==============================] - 175s 594ms/step - loss: 0.9627 - accuracy: 0.5177 - val_loss: 1.2